In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [4]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [5]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [6]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

In [7]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Dataset wine_quality downloaded and prepared to C:\Users\Godly PC\tensorflow_datasets\wine_quality\white\1.0.0. Subsequent calls will reuse this data.


In [9]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 22ms/step - loss: 42.9011 - root_mean_squared_error: 6.5499 - val_loss: 41.8143 - val_root_mean_squared_error: 6.4664
Epoch 2/100
17/17 [==============================] - 0s 6ms/step - loss: 40.8547 - root_mean_squared_error: 6.3918 - val_loss: 39.6044 - val_root_mean_squared_error: 6.2932
Epoch 3/100
17/17 [==============================] - 0s 5ms/step - loss: 39.1333 - root_mean_squared_error: 6.2557 - val_loss: 37.7918 - val_root_mean_squared_error: 6.1475
Epoch 4/100
17/17 [==============================] - 0s 5ms/step - loss: 37.4927 - root_mean_squared_error: 6.1231 - val_loss: 36.1507 - val_root_mean_squared_error: 6.0125
Epoch 5/100
17/17 [==============================] - 0s 5ms/step - loss: 35.9051 - root_mean_squared_error: 5.9921 - val_loss: 34.6192 - val_root_mean_squared_error: 5.8838
Epoch 6/100
17/17 [==============================] - 0s 6ms/step - loss: 34.3615 - root_mean_squared_error

Epoch 96/100
17/17 [==============================] - 0s 6ms/step - loss: 0.6201 - root_mean_squared_error: 0.7875 - val_loss: 0.6188 - val_root_mean_squared_error: 0.7867
Epoch 97/100
17/17 [==============================] - 0s 6ms/step - loss: 0.6154 - root_mean_squared_error: 0.7845 - val_loss: 0.6142 - val_root_mean_squared_error: 0.7837
Epoch 98/100
17/17 [==============================] - 0s 6ms/step - loss: 0.6116 - root_mean_squared_error: 0.7821 - val_loss: 0.6122 - val_root_mean_squared_error: 0.7824
Epoch 99/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6085 - root_mean_squared_error: 0.7800 - val_loss: 0.6084 - val_root_mean_squared_error: 0.7800
Epoch 100/100
17/17 [==============================] - 0s 6ms/step - loss: 0.6063 - root_mean_squared_error: 0.7786 - val_loss: 0.6051 - val_root_mean_squared_error: 0.7779
Model training finished.
Train RMSE: 0.777
Evaluating model performance...
Test RMSE: 0.778


In [10]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.3 - Actual: 5.0
Predicted: 5.3 - Actual: 5.0
Predicted: 5.5 - Actual: 6.0
Predicted: 6.2 - Actual: 6.0
Predicted: 5.6 - Actual: 6.0
Predicted: 5.7 - Actual: 7.0
Predicted: 5.6 - Actual: 5.0
Predicted: 6.5 - Actual: 7.0
Predicted: 5.5 - Actual: 6.0
Predicted: 5.6 - Actual: 5.0


In [ ]:
# Experiment 2: Bayesian neural network (BNN)

In [11]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [12]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [13]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 2s 220ms/step - loss: 30.8216 - root_mean_squared_error: 5.5501 - val_loss: 28.4313 - val_root_mean_squared_error: 5.3305
Epoch 2/500
5/5 [==============================] - 0s 19ms/step - loss: 29.3197 - root_mean_squared_error: 5.4135 - val_loss: 29.2697 - val_root_mean_squared_error: 5.4086
Epoch 3/500
5/5 [==============================] - 0s 19ms/step - loss: 28.2289 - root_mean_squared_error: 5.3116 - val_loss: 30.3357 - val_root_mean_squared_error: 5.5064
Epoch 4/500
5/5 [==============================] - 0s 19ms/step - loss: 29.6648 - root_mean_squared_error: 5.4452 - val_loss: 28.0813 - val_root_mean_squared_error: 5.2982
Epoch 5/500
5/5 [==============================] - 0s 18ms/step - loss: 29.6937 - root_mean_squared_error: 5.4477 - val_loss: 23.8818 - val_root_mean_squared_error: 4.8855
Epoch 6/500
5/5 [==============================] - 0s 18ms/step - loss: 26.5671 - root_mean_squared_error: 5.15

Epoch 96/500
5/5 [==============================] - 0s 19ms/step - loss: 2.9034 - root_mean_squared_error: 1.6978 - val_loss: 2.0905 - val_root_mean_squared_error: 1.4366
Epoch 97/500
5/5 [==============================] - 0s 19ms/step - loss: 2.8769 - root_mean_squared_error: 1.6889 - val_loss: 1.7665 - val_root_mean_squared_error: 1.3210
Epoch 98/500
5/5 [==============================] - 0s 19ms/step - loss: 2.2279 - root_mean_squared_error: 1.4844 - val_loss: 2.1306 - val_root_mean_squared_error: 1.4512
Epoch 99/500
5/5 [==============================] - 0s 19ms/step - loss: 2.1240 - root_mean_squared_error: 1.4490 - val_loss: 2.4864 - val_root_mean_squared_error: 1.5687
Epoch 100/500
5/5 [==============================] - 0s 19ms/step - loss: 1.6626 - root_mean_squared_error: 1.2804 - val_loss: 2.3355 - val_root_mean_squared_error: 1.5216
Epoch 101/500
5/5 [==============================] - 0s 19ms/step - loss: 2.8733 - root_mean_squared_error: 1.6886 - val_loss: 1.9455 - val_root

Epoch 144/500
5/5 [==============================] - 0s 19ms/step - loss: 0.8687 - root_mean_squared_error: 0.9168 - val_loss: 0.7789 - val_root_mean_squared_error: 0.8651
Epoch 145/500
5/5 [==============================] - 0s 20ms/step - loss: 0.7960 - root_mean_squared_error: 0.8752 - val_loss: 0.7943 - val_root_mean_squared_error: 0.8749
Epoch 146/500
5/5 [==============================] - 0s 19ms/step - loss: 0.7473 - root_mean_squared_error: 0.8476 - val_loss: 0.8029 - val_root_mean_squared_error: 0.8795
Epoch 147/500
5/5 [==============================] - 0s 19ms/step - loss: 0.7637 - root_mean_squared_error: 0.8581 - val_loss: 0.7775 - val_root_mean_squared_error: 0.8682
Epoch 148/500
5/5 [==============================] - 0s 18ms/step - loss: 0.8139 - root_mean_squared_error: 0.8882 - val_loss: 0.9371 - val_root_mean_squared_error: 0.9556
Epoch 149/500
5/5 [==============================] - 0s 20ms/step - loss: 0.9460 - root_mean_squared_error: 0.9577 - val_loss: 0.7880 - val_

Epoch 239/500
5/5 [==============================] - 0s 21ms/step - loss: 0.6946 - root_mean_squared_error: 0.8174 - val_loss: 0.7035 - val_root_mean_squared_error: 0.8243
Epoch 240/500
5/5 [==============================] - 0s 20ms/step - loss: 0.6639 - root_mean_squared_error: 0.7940 - val_loss: 0.7205 - val_root_mean_squared_error: 0.8325
Epoch 241/500
5/5 [==============================] - 0s 20ms/step - loss: 0.7519 - root_mean_squared_error: 0.8524 - val_loss: 0.7200 - val_root_mean_squared_error: 0.8320
Epoch 242/500
5/5 [==============================] - 0s 18ms/step - loss: 0.7336 - root_mean_squared_error: 0.8399 - val_loss: 0.6858 - val_root_mean_squared_error: 0.8095
Epoch 243/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6541 - root_mean_squared_error: 0.7896 - val_loss: 0.7007 - val_root_mean_squared_error: 0.8180
Epoch 244/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6760 - root_mean_squared_error: 0.8017 - val_loss: 0.6830 - val_

Epoch 334/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6839 - root_mean_squared_error: 0.8086 - val_loss: 0.6439 - val_root_mean_squared_error: 0.7832
Epoch 335/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6811 - root_mean_squared_error: 0.8077 - val_loss: 0.7130 - val_root_mean_squared_error: 0.8249
Epoch 336/500
5/5 [==============================] - 0s 18ms/step - loss: 0.6872 - root_mean_squared_error: 0.8090 - val_loss: 0.7154 - val_root_mean_squared_error: 0.8253
Epoch 337/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6908 - root_mean_squared_error: 0.8127 - val_loss: 0.6740 - val_root_mean_squared_error: 0.8000
Epoch 338/500
5/5 [==============================] - 0s 20ms/step - loss: 0.6586 - root_mean_squared_error: 0.7922 - val_loss: 0.6694 - val_root_mean_squared_error: 0.8008
Epoch 339/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6539 - root_mean_squared_error: 0.7885 - val_loss: 0.6885 - val_

Epoch 429/500
5/5 [==============================] - 0s 18ms/step - loss: 0.6675 - root_mean_squared_error: 0.7973 - val_loss: 0.6916 - val_root_mean_squared_error: 0.8125
Epoch 430/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6617 - root_mean_squared_error: 0.7925 - val_loss: 0.6907 - val_root_mean_squared_error: 0.8123
Epoch 431/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6462 - root_mean_squared_error: 0.7830 - val_loss: 0.7043 - val_root_mean_squared_error: 0.8185
Epoch 432/500
5/5 [==============================] - 0s 17ms/step - loss: 0.6881 - root_mean_squared_error: 0.8104 - val_loss: 0.6992 - val_root_mean_squared_error: 0.8157
Epoch 433/500
5/5 [==============================] - 0s 21ms/step - loss: 0.6367 - root_mean_squared_error: 0.7767 - val_loss: 0.6565 - val_root_mean_squared_error: 0.7878
Epoch 434/500
5/5 [==============================] - 0s 19ms/step - loss: 0.6337 - root_mean_squared_error: 0.7755 - val_loss: 0.7009 - val_

In [14]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.29, min: 4.49, max: 6.01, range: 1.53 - Actual: 5.0
Predictions mean: 5.29, min: 4.49, max: 6.01, range: 1.53 - Actual: 5.0
Predictions mean: 5.85, min: 5.31, max: 6.32, range: 1.01 - Actual: 6.0
Predictions mean: 6.17, min: 5.67, max: 6.42, range: 0.75 - Actual: 6.0
Predictions mean: 5.72, min: 5.14, max: 6.18, range: 1.04 - Actual: 6.0
Predictions mean: 5.82, min: 5.32, max: 6.19, range: 0.87 - Actual: 7.0
Predictions mean: 5.52, min: 4.94, max: 6.02, range: 1.08 - Actual: 5.0
Predictions mean: 6.28, min: 5.75, max: 6.5, range: 0.75 - Actual: 7.0
Predictions mean: 5.23, min: 4.55, max: 5.89, range: 1.34 - Actual: 6.0
Predictions mean: 5.42, min: 4.69, max: 5.97, range: 1.28 - Actual: 5.0


In [15]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 2s 45ms/step - loss: 41.4866 - root_mean_squared_error: 6.4407 - val_loss: 35.4223 - val_root_mean_squared_error: 5.9513
Epoch 2/500
17/17 [==============================] - 0s 6ms/step - loss: 40.9754 - root_mean_squared_error: 6.4009 - val_loss: 35.9837 - val_root_mean_squared_error: 5.9983
Epoch 3/500
17/17 [==============================] - 0s 6ms/step - loss: 37.3978 - root_mean_squared_error: 6.1150 - val_loss: 33.8187 - val_root_mean_squared_error: 5.8150
Epoch 4/500
17/17 [==============================] - 0s 6ms/step - loss: 36.0107 - root_mean_squared_error: 6.0005 - val_loss: 38.2910 - val_root_mean_squared_error: 6.1876
Epoch 5/500
17/17 [==============================] - 0s 6ms/step - loss: 33.6946 - root_mean_squared_error: 5.8043 - val_loss: 34.6480 - val_root_mean_squared_error: 5.8859
Epoch 6/500
17/17 [==============================] - 0s 6ms/step - loss: 33.3545 - root_mean_squared_error

Epoch 96/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6681 - root_mean_squared_error: 0.8104 - val_loss: 0.6462 - val_root_mean_squared_error: 0.7969
Epoch 97/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6719 - root_mean_squared_error: 0.8135 - val_loss: 0.6938 - val_root_mean_squared_error: 0.8261
Epoch 98/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6676 - root_mean_squared_error: 0.8105 - val_loss: 0.6568 - val_root_mean_squared_error: 0.8039
Epoch 99/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6874 - root_mean_squared_error: 0.8229 - val_loss: 0.6637 - val_root_mean_squared_error: 0.8083
Epoch 100/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6712 - root_mean_squared_error: 0.8126 - val_loss: 0.6662 - val_root_mean_squared_error: 0.8103
Epoch 101/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6819 - root_mean_squared_error: 0.8195 - val_loss: 0.6649 - va

17/17 [==============================] - 0s 7ms/step - loss: 0.6280 - root_mean_squared_error: 0.7848 - val_loss: 0.6118 - val_root_mean_squared_error: 0.7735
Epoch 191/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6413 - root_mean_squared_error: 0.7933 - val_loss: 0.6153 - val_root_mean_squared_error: 0.7759
Epoch 192/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6452 - root_mean_squared_error: 0.7957 - val_loss: 0.6080 - val_root_mean_squared_error: 0.7711
Epoch 193/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6317 - root_mean_squared_error: 0.7874 - val_loss: 0.6299 - val_root_mean_squared_error: 0.7867
Epoch 194/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6312 - root_mean_squared_error: 0.7869 - val_loss: 0.6240 - val_root_mean_squared_error: 0.7824
Epoch 195/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6239 - root_mean_squared_error: 0.7821 - val_loss: 0.6273 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.6320 - root_mean_squared_error: 0.7868 - val_loss: 0.6018 - val_root_mean_squared_error: 0.7670
Epoch 285/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6105 - root_mean_squared_error: 0.7725 - val_loss: 0.6146 - val_root_mean_squared_error: 0.7760
Epoch 286/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6182 - root_mean_squared_error: 0.7787 - val_loss: 0.5935 - val_root_mean_squared_error: 0.7617
Epoch 287/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6113 - root_mean_squared_error: 0.7735 - val_loss: 0.6045 - val_root_mean_squared_error: 0.7686
Epoch 288/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6076 - root_mean_squared_error: 0.7706 - val_loss: 0.5954 - val_root_mean_squared_error: 0.7635
Epoch 289/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6061 - root_mean_squared_error: 0.7701 - val_loss: 0.6188 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.6015 - root_mean_squared_error: 0.7670 - val_loss: 0.6113 - val_root_mean_squared_error: 0.7729
Epoch 379/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6037 - root_mean_squared_error: 0.7678 - val_loss: 0.5890 - val_root_mean_squared_error: 0.7589
Epoch 380/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6119 - root_mean_squared_error: 0.7732 - val_loss: 0.5925 - val_root_mean_squared_error: 0.7612
Epoch 381/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5941 - root_mean_squared_error: 0.7616 - val_loss: 0.6321 - val_root_mean_squared_error: 0.7862
Epoch 382/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6114 - root_mean_squared_error: 0.7732 - val_loss: 0.6024 - val_root_mean_squared_error: 0.7665
Epoch 383/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6157 - root_mean_squared_error: 0.7761 - val_loss: 0.6001 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.6061 - root_mean_squared_error: 0.7692 - val_loss: 0.6081 - val_root_mean_squared_error: 0.7701
Epoch 473/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6049 - root_mean_squared_error: 0.7685 - val_loss: 0.5909 - val_root_mean_squared_error: 0.7594
Epoch 474/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6169 - root_mean_squared_error: 0.7758 - val_loss: 0.5944 - val_root_mean_squared_error: 0.7614
Epoch 475/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6028 - root_mean_squared_error: 0.7666 - val_loss: 0.6111 - val_root_mean_squared_error: 0.7718
Epoch 476/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6002 - root_mean_squared_error: 0.7653 - val_loss: 0.5950 - val_root_mean_squared_error: 0.7614
Epoch 477/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6023 - root_mean_squared_error: 0.7666 - val_loss: 0.6045 - val_root_mea

In [ ]:
#Experiment 3: probabilistic Bayesian neural network

In [16]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [17]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 2s 39ms/step - loss: 132.9582 - root_mean_squared_error: 5.5097 - val_loss: 128.1857 - val_root_mean_squared_error: 5.8051
Epoch 2/1000
17/17 [==============================] - 0s 6ms/step - loss: 138.2231 - root_mean_squared_error: 5.5848 - val_loss: 61.0383 - val_root_mean_squared_error: 5.3972
Epoch 3/1000
17/17 [==============================] - 0s 7ms/step - loss: 92.7934 - root_mean_squared_error: 5.4470 - val_loss: 106.1828 - val_root_mean_squared_error: 5.4766
Epoch 4/1000
17/17 [==============================] - 0s 6ms/step - loss: 105.8423 - root_mean_squared_error: 5.3809 - val_loss: 97.3540 - val_root_mean_squared_error: 5.2679
Epoch 5/1000
17/17 [==============================] - 0s 6ms/step - loss: 70.7343 - root_mean_squared_error: 5.3456 - val_loss: 71.9408 - val_root_mean_squared_error: 4.8879
Epoch 6/1000
17/17 [==============================] - 0s 6ms/step - loss: 57.5978 - root_mean_sq

Epoch 48/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.7926 - root_mean_squared_error: 2.2797 - val_loss: 1.7439 - val_root_mean_squared_error: 2.2732
Epoch 49/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.7013 - root_mean_squared_error: 2.1069 - val_loss: 1.6856 - val_root_mean_squared_error: 2.0197
Epoch 50/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.6520 - root_mean_squared_error: 2.0105 - val_loss: 1.6399 - val_root_mean_squared_error: 1.9568
Epoch 51/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.5949 - root_mean_squared_error: 1.9038 - val_loss: 1.5090 - val_root_mean_squared_error: 1.7053
Epoch 52/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.5498 - root_mean_squared_error: 1.8247 - val_loss: 1.5461 - val_root_mean_squared_error: 1.8710
Epoch 53/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.5529 - root_mean_squared_error: 1.7714 - val_loss: 1.4197 

17/17 [==============================] - 0s 6ms/step - loss: 1.1974 - root_mean_squared_error: 1.1491 - val_loss: 1.2352 - val_root_mean_squared_error: 1.1348
Epoch 143/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.2090 - root_mean_squared_error: 1.1227 - val_loss: 1.2304 - val_root_mean_squared_error: 1.0781
Epoch 144/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.2114 - root_mean_squared_error: 1.1561 - val_loss: 1.1921 - val_root_mean_squared_error: 1.1347
Epoch 145/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.2318 - root_mean_squared_error: 1.1541 - val_loss: 1.2392 - val_root_mean_squared_error: 1.2000
Epoch 146/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.2064 - root_mean_squared_error: 1.1562 - val_loss: 1.2176 - val_root_mean_squared_error: 1.1554
Epoch 147/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.2130 - root_mean_squared_error: 1.1315 - val_loss: 1.2759 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1795 - root_mean_squared_error: 1.0970 - val_loss: 1.1632 - val_root_mean_squared_error: 1.0762
Epoch 237/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1839 - root_mean_squared_error: 1.1162 - val_loss: 1.1724 - val_root_mean_squared_error: 1.1087
Epoch 238/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1704 - root_mean_squared_error: 1.0916 - val_loss: 1.1858 - val_root_mean_squared_error: 1.1292
Epoch 239/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1792 - root_mean_squared_error: 1.0884 - val_loss: 1.1750 - val_root_mean_squared_error: 1.1031
Epoch 240/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1732 - root_mean_squared_error: 1.1152 - val_loss: 1.2025 - val_root_mean_squared_error: 1.0675
Epoch 241/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1749 - root_mean_squared_error: 1.1051 - val_loss: 1.1680 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1842 - root_mean_squared_error: 1.0917 - val_loss: 1.1742 - val_root_mean_squared_error: 1.0456
Epoch 331/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1786 - root_mean_squared_error: 1.0990 - val_loss: 1.1450 - val_root_mean_squared_error: 1.0354
Epoch 332/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1707 - root_mean_squared_error: 1.1075 - val_loss: 1.1740 - val_root_mean_squared_error: 1.0244
Epoch 333/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1662 - root_mean_squared_error: 1.0743 - val_loss: 1.1698 - val_root_mean_squared_error: 1.0629
Epoch 334/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1790 - root_mean_squared_error: 1.1096 - val_loss: 1.1628 - val_root_mean_squared_error: 1.0646
Epoch 335/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1783 - root_mean_squared_error: 1.0837 - val_loss: 1.1870 - val_roo

...

17/17 [==============================] - 0s 6ms/step - loss: 1.1585 - root_mean_squared_error: 1.0936 - val_loss: 1.1665 - val_root_mean_squared_error: 1.0862
Epoch 613/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1631 - root_mean_squared_error: 1.0601 - val_loss: 1.1538 - val_root_mean_squared_error: 1.1025
Epoch 614/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1601 - root_mean_squared_error: 1.0628 - val_loss: 1.1554 - val_root_mean_squared_error: 1.0790
Epoch 615/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1603 - root_mean_squared_error: 1.0970 - val_loss: 1.1473 - val_root_mean_squared_error: 1.1652
Epoch 616/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1639 - root_mean_squared_error: 1.0876 - val_loss: 1.1668 - val_root_mean_squared_error: 1.1142
Epoch 617/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1617 - root_mean_squared_error: 1.0743 - val_loss: 1.1615 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1701 - root_mean_squared_error: 1.0997 - val_loss: 1.1497 - val_root_mean_squared_error: 1.0748
Epoch 707/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1654 - root_mean_squared_error: 1.0769 - val_loss: 1.1487 - val_root_mean_squared_error: 1.0392
Epoch 708/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1625 - root_mean_squared_error: 1.0729 - val_loss: 1.1576 - val_root_mean_squared_error: 1.0729
Epoch 709/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1567 - root_mean_squared_error: 1.1100 - val_loss: 1.1589 - val_root_mean_squared_error: 1.0939
Epoch 710/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1588 - root_mean_squared_error: 1.0779 - val_loss: 1.1402 - val_root_mean_squared_error: 1.0509
Epoch 711/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1566 - root_mean_squared_error: 1.0726 - val_loss: 1.1561 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1526 - root_mean_squared_error: 1.0879 - val_loss: 1.1764 - val_root_mean_squared_error: 1.0841
Epoch 801/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1577 - root_mean_squared_error: 1.0839 - val_loss: 1.1726 - val_root_mean_squared_error: 1.0580
Epoch 802/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1485 - root_mean_squared_error: 1.0838 - val_loss: 1.1492 - val_root_mean_squared_error: 1.0878
Epoch 803/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1557 - root_mean_squared_error: 1.0592 - val_loss: 1.1501 - val_root_mean_squared_error: 1.1021
Epoch 804/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1591 - root_mean_squared_error: 1.0751 - val_loss: 1.1511 - val_root_mean_squared_error: 1.1041
Epoch 805/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1544 - root_mean_squared_error: 1.0778 - val_loss: 1.1482 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1493 - root_mean_squared_error: 1.0537 - val_loss: 1.1404 - val_root_mean_squared_error: 1.0467
Epoch 895/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1462 - root_mean_squared_error: 1.0661 - val_loss: 1.1486 - val_root_mean_squared_error: 1.0218
Epoch 896/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1444 - root_mean_squared_error: 1.0455 - val_loss: 1.1476 - val_root_mean_squared_error: 1.0612
Epoch 897/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1451 - root_mean_squared_error: 1.0557 - val_loss: 1.1523 - val_root_mean_squared_error: 1.0760
Epoch 898/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1514 - root_mean_squared_error: 1.0664 - val_loss: 1.1577 - val_root_mean_squared_error: 1.0415
Epoch 899/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1559 - root_mean_squared_error: 1.0790 - val_loss: 1.1454 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1485 - root_mean_squared_error: 1.0779 - val_loss: 1.1550 - val_root_mean_squared_error: 1.0743
Epoch 989/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1466 - root_mean_squared_error: 1.0581 - val_loss: 1.1501 - val_root_mean_squared_error: 1.0501
Epoch 990/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1457 - root_mean_squared_error: 1.0784 - val_loss: 1.1535 - val_root_mean_squared_error: 1.0529
Epoch 991/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1503 - root_mean_squared_error: 1.0716 - val_loss: 1.1572 - val_root_mean_squared_error: 1.0531
Epoch 992/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1486 - root_mean_squared_error: 1.0516 - val_loss: 1.1468 - val_root_mean_squared_error: 1.0574
Epoch 993/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1540 - root_mean_squared_error: 1.0802 - val_loss: 1.1562 - val_roo

In [18]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.26, stddev: 0.63, 95% CI: [6.51 - 4.02] - Actual: 5.0
Prediction mean: 5.26, stddev: 0.63, 95% CI: [6.51 - 4.02] - Actual: 5.0
Prediction mean: 5.86, stddev: 0.67, 95% CI: [7.17 - 4.56] - Actual: 6.0
Prediction mean: 6.53, stddev: 0.83, 95% CI: [8.17 - 4.9] - Actual: 6.0
Prediction mean: 5.92, stddev: 0.68, 95% CI: [7.25 - 4.59] - Actual: 6.0
Prediction mean: 5.89, stddev: 0.78, 95% CI: [7.42 - 4.36] - Actual: 7.0
Prediction mean: 5.42, stddev: 0.66, 95% CI: [6.71 - 4.12] - Actual: 5.0
Prediction mean: 6.62, stddev: 0.85, 95% CI: [8.28 - 4.96] - Actual: 7.0
Prediction mean: 5.26, stddev: 0.64, 95% CI: [6.51 - 4.02] - Actual: 6.0
Prediction mean: 5.31, stddev: 0.68, 95% CI: [6.64 - 3.98] - Actual: 5.0
